## Newton para Sistemas não Lineares

O método de Newton para sistemas não lineares busca encontrar a solução de um sistema f(x) = 0 utilizando aproximações sucessivas e a inversa do Jacobiano.


In [3]:
import numpy as np
import sympy as sp
from tabulate import tabulate

In [5]:
def newton_sistemas(funcs, vars, x0, tol):
    """
    Resolve sistemas de equações não lineares pelo método de Newton.
    
    Parâmetros:
    funcs -- Lista de funções do sistema
    vars -- Lista de variáveis
    x0 -- Aproximação inicial (array)
    tol -- Erro de tolerância
    
    Retorna:
    Solução aproximada e tabela com as iterações
    """
    # Converte as funções simbólicas para funções numéricas
    J = sp.Matrix([[sp.diff(f, var) for var in vars] for f in funcs])  # Jacobiano
    J_inv = J.inv()  # Inversa do Jacobiano
    
    f_lambdify = sp.lambdify(vars, funcs, 'numpy')
    J_inv_lambdify = sp.lambdify(vars, J_inv, 'numpy')
    
    x = np.array(x0, dtype=float)
    tabela = []
    iter = 0
    
    while iter <= 10:
        F_val = np.array(f_lambdify(*x), dtype=float).flatten()
        J_inv_val = np.array(J_inv_lambdify(*x), dtype=float)
        
        delta_x = -J_inv_val @ F_val
        x_new = x + delta_x
        
        erro_absoluto = round(np.linalg.norm(delta_x), 4)
        erro_relativo = round(erro_absoluto / (np.linalg.norm(x_new) if np.linalg.norm(x_new) != 0 else 1), 4)
        
        tabela.append([iter, *x, erro_absoluto, erro_relativo])
        iter += 1
        
        # if erro_absoluto <= tol:
            # return np.round(x_new, 4), tabela
        
        x = x_new
    
    return np.round(x, 4), tabela

In [11]:
# Entrada do usuário
n = int(input("Digite o número de equações/variáveis: "))
vars = sp.symbols(f'x1:{n+1}')
funcs = [sp.sympify("-10*x1+(25*cos(x1))+10*x2-10*x1"), sp.sympify("-10*x2+10*x1+(15*cos(x2))+10*x3-10*x2"), sp.sympify("10*x3-10*x2-(5*cos(x3))")]

# print("Digite as funções do sistema:")
# for i in range(n):
    # funcs.append(sp.sympify(input(f"f{i+1}(x1, ..., xn) = ")))

x0 = [float(input(f"Digite x{i+1} inicial: ")) for i in range(n)]
tol = float(input("Digite o erro de tolerância: "))

solucao, tabela = newton_sistemas(funcs, vars, x0, tol)

# Exibindo tabela de resultados
headers = ["Iteração"] + [f"x{i+1}" for i in range(n)] + ["Erro Absoluto", "Erro Relativo"]
print(tabulate(tabela, headers=headers, floatfmt=".4f"))
print(f"Solução aproximada: {solucao}")

  Iteração      x1      x2      x3    Erro Absoluto    Erro Relativo
----------  ------  ------  ------  ---------------  ---------------
         0  1.0000  1.0000  1.0000           0.7286           0.2997
         1  1.1998  1.4692  1.5204           0.0546           0.0230
         2  1.1863  1.4349  1.4802           0.0001           0.0000
         3  1.1863  1.4349  1.4801           0.0000           0.0000
         4  1.1863  1.4349  1.4801           0.0000           0.0000
         5  1.1863  1.4349  1.4801           0.0000           0.0000
         6  1.1863  1.4349  1.4801           0.0000           0.0000
         7  1.1863  1.4349  1.4801           0.0000           0.0000
         8  1.1863  1.4349  1.4801           0.0000           0.0000
         9  1.1863  1.4349  1.4801           0.0000           0.0000
        10  1.1863  1.4349  1.4801           0.0000           0.0000
Solução aproximada: [1.1863 1.4349 1.4801]
